# **Loading Libraries**

In [ ]:
# EDA & Text Processing
import numpy as np
import pandas as pd
import re
# Model Building
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
# DL model imports
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import BertTokenizer, TFBertModel
from transformers import pipeline

# **Exploring Dataframe**

In [ ]:
df = pd.read_csv('/content/train_E6oV3lV-3.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [ ]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


## **Text Processing**

In [ ]:
def clean_text(text):
    text = re.sub(r'@[\w]*', '', text)  # subbing out mentions
    text = re.sub(r'#', '', text)  # subbing out hashtags
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # subbing out URLs
    text = re.sub(r'[^A-Za-z\s]', '', text)  # subbing out special characters
    text = text.lower()  # Converting to lowercase
    return text

In [ ]:
df['clean_text'] = df['tweet'].apply(clean_text)

In [ ]:
df['clean_text'].head()

,clean_text
0,when a father is dysfunctional and is so sel...
1,thanks for lyft credit i cant use cause they...
2,bihday your majesty
3,model i love u take with u all the time in u...
4,factsguide society now motivation


In [ ]:
df['clean_text'].str.len().max()

139

## Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=10)

# **Frequency-based Text Embedding**

  > ### Bag of Words

In [ ]:
# Bag of words
bow_vectorizer = CountVectorizer(max_features = 10000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

    > Creating Pipeline and Hyperparameter Tuning

In [ ]:
bow_pipe = Pipeline([
    ('classifier', LogisticRegression())
])

param_grid = [
    # Logistic Regression
    {'classifier': [LogisticRegression(max_iter=2000)],
     'classifier__C': np.logspace(-3, 3, 15),
     'classifier__penalty': ['l1', 'l2'],
     'classifier__solver': ['liblinear']},
    # RandomForestClassifier
    {'classifier': [RandomForestClassifier()],
     'classifier__n_estimators': [50, 100],
     'classifier__max_features': ['sqrt', 'log2'],
     'classifier__max_depth': [None, 10, 20]}
]

In [ ]:
results = pd.DataFrame(columns=['Model', 'Training Accuracy', 'Test Accuracy'])

for params in param_grid:
  # setup Pipeline with the current Classifier
  bow_pipe.set_params(**{k: v[0] for k, v in params.items()})
  grid_search = GridSearchCV(bow_pipe, param_grid= [params], cv=10, scoring='accuracy', n_jobs=-1)
  grid_search.fit(X_train_bow, y_train)

  # extracting the best metrics and evaluating current Classifier
  best_model = grid_search.best_estimator_
  train_acc = best_model.score(X_train_bow, y_train)*100
  test_acc = best_model.score(X_test_bow, y_test)*100

  # recording findings into a temporary df
  temp_df = pd.DataFrame({
        'Model': [type(best_model.named_steps['classifier']).__name__ + '_bow'],
        'Training Accuracy': [train_acc],
        'Test Accuracy': [test_acc]
    })

  #finalising entry into the findings df
  results = pd.concat([results, temp_df], ignore_index=True)

<ipython-input-13-d9195defe0cf>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, temp_df], ignore_index=True)


  > ### TF-IDF Vectorizor

In [ ]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features = 10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

    > Creating Pipeline and Hyperparameter Tuning

In [ ]:
tfidf_vectorizer_pipe = Pipeline([
    ('classifier', LogisticRegression())
])

param_grid = [
    # Logistic Regression
    {'classifier': [LogisticRegression(max_iter=2000)],
     'classifier__C': np.logspace(-3, 3, 15),
     'classifier__penalty': ['l1', 'l2'],
     'classifier__solver': ['liblinear']},
    # RandomForestClassifier
    {'classifier': [RandomForestClassifier()],
     'classifier__n_estimators': [50, 100],
     'classifier__max_features': ['sqrt', 'log2'],
     'classifier__max_depth': [None, 10, 20]}
]

In [ ]:
for params in param_grid:
  # setup Pipeline with the current Classifier
  tfidf_vectorizer_pipe.set_params(**{k: v[0] for k, v in params.items()})
  grid_search = GridSearchCV(tfidf_vectorizer_pipe, param_grid= [params], cv=10, scoring='accuracy', n_jobs=-1)
  grid_search.fit(X_train_bow, y_train)

  # extracting the best metrics and evaluating current Classifier
  best_model = grid_search.best_estimator_
  train_acc = best_model.score(X_train_tfidf, y_train)*100
  test_acc = best_model.score(X_test_tfidf, y_test)*100

  # recording findings into a temporary df
  temp_df = pd.DataFrame({
        'Model': [type(best_model.named_steps['classifier']).__name__ + '_tfidf'],
        'Training Accuracy': [train_acc],
        'Test Accuracy': [test_acc]
    })

  #finalising entry into the findings df
  results = pd.concat([results, temp_df], ignore_index=True)

# **Semantic & Contextual Text Embedding Techniques**


> # roBERTa
  The RoBERTa-base model is trained on ~124M tweets from January 2018 to December 2021, and finetuned for sentiment analysis with the TweetEval benchmark. For further details on the model [click here](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt= True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier= pipeline('sentiment-analysis', model=model, tokenizer= tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
classifier('amazing!')[0]['label']

'positive'

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt= True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1

      > Mapping Default Model labels to corresponding Target Encoded values in binary.

In [ ]:
def sentiment_mapper(sentiment_label):
  """
  > "LABEL_0" == 'negative'(hate)  tweet
  > "LABEL_1" == 'neutral' tweet
  > "LABEL_2" == 'positive' tweet
  """

  if sentiment_label ==  'LABEL_0':
    return 1
  else:
    return 0

In [ ]:
def batch_predict(data, batch_size=761):
    label_vals = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]

        # Tokenizing the batch
        inputs = tokenizer(batch, return_tensors="tf", padding=True, truncation=True, max_length=512)

        # Running predictions for the batch
        outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
        probs = tf.nn.softmax(outputs.logits, axis=-1)
        preds = np.argmax(probs, axis=1)

        # Converting model predictions to binary
        labels = [sentiment_mapper(f'LABEL_{p}') for p in preds]
        label_vals.extend(labels)
    return label_vals

In [ ]:
X_train.dtypes

dtype('O')

In [ ]:
train_preds = batch_predict(X_train.astype(str).tolist())
test_preds = batch_predict(X_test.astype(str).tolist())

train_acc = accuracy_score(train_preds, y_train) * 100
test_acc = accuracy_score(test_preds, y_test) * 100

temp_df = pd.DataFrame({
    'Model': ['roBERTa'],
    'Training Accuracy': [train_acc],
    'Test Accuracy': [test_acc]
})

In [ ]:
results = pd.concat([results, temp_df], ignore_index=True)

  > # Universal Sentence Encoder
   The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language tasks.The model is trained and optimized for greater-than-word length text, such as sentences, phrases or short paragraphs. It is trained on a variety of data sources and a variety of tasks with the aim of dynamically accommodating a wide variety of natural language understanding tasks.
   For more details on the model [click here](https://www.kaggle.com/models/google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder/2?tfhub-redirect=true).

In [ ]:
url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(url)

In [ ]:
def get_embeddings(data):
  return embed(data).numpy()

In [ ]:
X_train_embeddings = get_embeddings(X_train)
X_test_embeddings = get_embeddings(X_test)

      > Training a Logistic Regression and Random Forest model on USE.

In [ ]:

logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_embeddings, y_train)


train_acc_logistic = logistic_model.score(X_train_embeddings, y_train) * 100
test_acc_logistic = logistic_model.score(X_test_embeddings, y_test) * 100

print(f'Logistic Regression - Training Accuracy: {train_acc_logistic:.2f}%')
print(f'Logistic Regression - Test Accuracy: {test_acc_logistic:.2f}%')


Logistic Regression - Training Accuracy: 95.56%
Logistic Regression - Test Accuracy: 95.09%


In [ ]:

rf_model = RandomForestClassifier()
rf_model.fit(X_train_embeddings, y_train)


train_acc_rf = rf_model.score(X_train_embeddings, y_train) * 100
test_acc_rf = rf_model.score(X_test_embeddings, y_test) * 100

print(f'Random Forest - Training Accuracy: {train_acc_rf:.2f}%')
print(f'Random Forest - Test Accuracy: {test_acc_rf:.2f}%')

Random Forest - Training Accuracy: 100.00%
Random Forest - Test Accuracy: 95.34%


In [ ]:
temp_df = pd.DataFrame({
    'Model': ['LogisticRegression_USE'],
    'Training Accuracy': [train_acc_logistic],
    'Test Accuracy': [test_acc_logistic]
})

results = pd.concat([results, temp_df], ignore_index=True)

In [ ]:
temp_df = pd.DataFrame({
    'Model': ['RamdomForestClassifier_USE'],
    'Training Accuracy': [train_acc_rf],
    'Test Accuracy': [test_acc_rf]
})

results = pd.concat([results, temp_df], ignore_index=True)

# **Final Result**

In [ ]:
results

,Model,Training Accuracy,Test Accuracy
0,LogisticRegression_bow,98.936212,96.120757
1,RandomForestClassifier_bow,99.980445,96.433599
2,LogisticRegression_tfidf,93.261371,92.992335
3,RandomForestClassifier_tfidf,93.464742,93.007977
4,roBERTa,78.008526,78.398248
5,LogisticRegression_USE,95.561031,95.088378
6,RamdomForestClassifier_USE,99.996089,95.338652
